# Piping a RunnableParallel with Other Runnables

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [ ]:
pip show langchain

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [ ]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
     '''
     I'm an intermediate level programmer.
     
     Consider the following literature:
     {books}
     
     Also, consider the following projects:
     {projects}
     
     Roughly how much time would it take me to complete the literature and the projects?
     
     '''
)

In [ ]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 500)

In [ ]:
string_parser = StrOutputParser()

In [ ]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [ ]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [ ]:
chain_parallel.invoke({'programming language':'Python'})

In [ ]:
chain_time1 = (RunnableParallel({'books':chain_books, 
                                'projects':chain_projects}) 
              | chat_template_time 
              | chat 
              | string_parser
             )

In [ ]:
chain_time2 = ({'books':chain_books, 
                'projects':chain_projects}
              | chat_template_time 
              | chat 
              | string_parser
             )

In [ ]:
print(chain_time2.invoke({'programming language':'Python'}))

In [ ]:
chain_time2.get_graph().print_ascii()